In [ ]:
#!pip install -qq git+https://github.com/openai/whisper.git 
#!pip install --force-reinstall git+https://github.com/mattiaswickberg/whisperx.git

In [ ]:
#!pip install -qq torch==1.8.1 torchvision==0.9.1 torchaudio==0.9.0 torchtext==0.8.0
#!pip install -qq speechbrain==0.5.12
#!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
#!pip install pydub

In [ ]:
import subprocess #if stripping audio from video file
import datetime #to print time of start and end of analysis
#import torch
#import torchvision

In [ ]:
import json

f = open('../credentials.json')
credentials = json.load(f)
hf_token = credentials["hf_token"]
wav2vec_model = credentials["wav2vec_large"]

In [ ]:
import re
def speakers(file):
    speakers = re.findall("n_(\d+).", file)
    if len(speakers) > 0:
        return int(speakers[0])
    else: 
        return 1

In [ ]:
def isolate_audio(file):
    wav_file = file + '.wav'
    subprocess.call(['ffmpeg', '-i', file, wav_file, '-y'])
    return wav_file

In [ ]:
def transcribe(wav_file, num_speakers):
    !whisperx {wav_file} --model large --language sv --output_type txt --hf_token {hf_token} --min_speakers {num_speakers} --vad_filter --align_model {wav2vec_model}


In [ ]:
from pydub import AudioSegment

def file_length(file):
    # Load the audio file using pydub
    audio_file = AudioSegment.from_file(file, format="wav")

    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)

    # Convert the duration from milliseconds to seconds
    duration = datetime.timedelta(milliseconds=duration_ms)
    duration_str = str(duration)
    
    return duration_str

In [ ]:
def log_file(text):
    with open('log_file.txt', 'a') as file:
        s = "".join(text)
        file.write(s)

In [ ]:
import os

def iterate():
    try:
        
        for filename in os.listdir("/mimer/NOBACKUP/groups/stls/Data"):
            if not ".ipynb" in filename:
                start_1 = start = datetime.datetime.now()
                file_path = os.path.join("/mimer/NOBACKUP/groups/stls/Data", filename)
                num_speakers = speakers(file_path)
                wav_file = isolate_audio(file_path)
                transcribe(wav_file, num_speakers)
                end_1 = start = datetime.datetime.now()
                delta_1 = end_1 -  start_1
                log_file("Duration of " + filename + "is " + file_length(wav_file))
                log_file("Transkription for " + filename + " was " + str(delta_1) + '\n')
            
    except Exception as e:
            print(f'An error has occured: {e}')
        

In [ ]:
start = datetime.datetime.now()

iterate()

end = datetime.datetime.now()
delta = end - start
log_file("Runtime for whole batch was " + str(delta) +('\n'))